In [37]:
import telebot
import datetime
import sqlite3
from sqlite3 import Error
import os
import csv
from IPython.display import clear_output
from tqdm import tqdm
from time import time
import uule_grabber
import xlsxwriter
import json
import secrets
from seleniumwire import webdriver
import time
import random
import re


bot = telebot.TeleBot('1724763898:AAGv3hirMnXsi7d0Yqur7Cr1QamJmqGmbiI')
chatId = 449236929

useragent = 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36'

filekeys = ''
# filename = fil
inindex =[]
zeroresult = []
proxies = []
# noindex = []

# read proxy list
fileproxy = 'Proxy_HTTP.txt'
def listOfproxy(fileproxy):
    global PROXY_USER
    global PROXY_PASS
    global PROXY_PORT
    with open(fileproxy, 'r', encoding="utf-8") as f:
        proxies1 = list(filter(lambda x: len(x.strip()) > 0, f))
    proxies2 = list(map(lambda x: x.replace('\n', '').replace(' ', ''), proxies1))
    pro = {}
    for g in proxies2:
        k = g.split(':')[0]
        v = g.split(':')[1]
        pro[k] = v
    PROXY_HTTP = pro.pop('ProxyType')
    PROXY_USER = pro.pop('Login')  # username
    PROXY_PASS = pro.pop('Password') # password tipfoto
    proxies = list(map(lambda x: x, pro.keys()))
    PROXY_HOST = secrets.choice(proxies)
    PROXY_PORT = int(list(map(lambda x: x, pro.values()))[0])
    # print(PROXY_HOST)
    return proxies


# сохраняем массив собраных урлов        
def savesqllist(inindex):
#     print(inindex)
    for f in inindex:
        query = f[0]
        url_filter = f[1]
#         score = f[2]
        score = inindex.index(f)+1
        create_top = f"""
        INSERT INTO
          top (keywd, url, score)
        VALUES
          ('{query}', '{url_filter}', '{score}');
        """

        execute_query(connection, create_top) 
#     clear_output(wait=True)
#         print('sql - Done')

# сохраняем фразу если ошибка
def saveerr(query, score):
        create_err = f"""
        INSERT OR IGNORE INTO
          errkeys (errkey, score)
        VALUES
          ('{query}', '{score}');
        """

        execute_query(connection, create_err)         


#get_chromedriver Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36
def get_chromedriver(use_proxy=False, user_agent=None):
    global PROXY_HOST
    global proxies
#     useragent_random = useragent_r()
#     print(useragent_random)
    useragent_random = useragent
    path = os.path.dirname(os.path.abspath(r"C:\Users\Lids1.SERVER\Desktop\Python\selenium\chrome1\chromedriver"))
#     path = os.path.dirname(os.path.abspath(r"C:\Users\Admin\Documents\Python\selenium\chrome\chromedriver"))
#     path = os.path.dirname(os.path.abspath(r"C:\Users\Lids1\Desktop\SEO\Python\selenium\chrome\chromedriver"))
    if len(proxies) < 3:
        proxies = listOfproxy(fileproxy)
#         proxies = list(map(lambda x: x, pro.keys()))
    PROXY_HOST = secrets.choice(proxies)
    chrome_options = webdriver.ChromeOptions()
    chrome_options.headless = True
    # disable webdriver mode

    # for ChromeDriver version 79.0.3945.16 or over
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")

#     chrome_options.add_argument(f'user-agent={useragent}')
    #chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    #chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_argument(f"user-agent={useragent_random}")
    
    proxy_options = {
        'proxy': {
            'https': f'https://{PROXY_USER}:{PROXY_PASS}@{PROXY_HOST}:{PROXY_PORT}'
        }
    }


    driver = webdriver.Chrome(
        os.path.join(path, 'chromedriver'),
        seleniumwire_options=proxy_options,
        options=chrome_options)
#     print (PROXY_HOST)
    return driver

def get_driver(query, score):
    
    driver = get_chromedriver(use_proxy=True)
    try:
        queryl = re.sub('\s','%20',query)
        takeurl = f'https://www.google.{zone}/search?q={queryl}&pws=0&uule={uule}&hl={hl}&gl={country}&ip=0.0.0.0&source_ip=0.0.0.0&num={num}'
#         takeurl = f'https://www.google.com.ua/webhp?hl={hl}&uule={uule}&gl={country}&ip=0.0.0.0&source_ip=0.0.0.0&num={num}'
        driver.get(takeurl)
        time.sleep(1)
        # Проверяем если капча
        result_search = driver.find_elements_by_xpath("//form[@id='captcha-form']")
        if (result_search):
            print ('Reboot proxy')
            proxies.remove(PROXY_HOST)
            time.sleep(1)
            driver.close()
            driver.quit()
#             try: os.system("taskkill /im chromedriver.exe")
#             except: pass
            get_driver(query, score)
        else:
            # Проверяем если нет результатов в выдаче
            try:
                result_num = driver.find_elements_by_xpath("//div[@id='result-stats']")[0]
                if result_num.text.split(' ')[2] == '0': return
            except:pass
            try:
                result_num = driver.find_element_by_xpath("//div[@class='card-section']")
                if len(result_num.text.split(' ')[:2]) == 2: return
            except:pass
            print('parsing')
            try:
                result_stats = driver.find_elements_by_xpath("//div[@class='yuRUbf']/a")
                result_asc = driver.find_element_by_xpath("//div[@class='Wt5Tfe']")
                result_asc2 = result_asc.find_elements_by_xpath("//div[@jsname='Cpkphb']")
                result_asc1 = result_asc.find_elements_by_xpath("//div[@class='yuRUbf']/a")
            except:
                print('ошибка сбора данных')
                pass
#             result_stats = driver.find_elements_by_xpath("//div[@class='r']/a")
            print('Обробка')
            inindex.clear()
            zeroresult.clear()
            print(len(result_stats))
            if (len(result_stats) == 0):        
                print ('Пусто')
                time.sleep(1)
                saveerr(query, score)
                return

            else:
                asc_urls = []
                f = 0
                try:
                    print('Також питають: ', len(result_asc2))
                    for links in result_asc1[1:]:
                        url_filter = links.get_attribute('href')
                        if 'translate' in url_filter: continue
                        if url_filter not in asc_urls:
                            f = f +1
                            asc_urls.append(url_filter)
                            print(url_filter)
                            if f == len(result_asc2): break
                except:
                    print('помилка asc')
                    pass
                all_urls = []
                print('Органічний пошук')
                for links in result_stats:
                    url_filter = links.get_attribute('href')
                #     if 'translate' in url_filter or 'google' in url_filter: continue
                    if 'translate' in url_filter: continue
                    all_urls.append(url_filter)
                    if url_filter in asc_urls:
                        all_urls.remove(url_filter)
                        asc_urls.remove(url_filter)
                print(len(all_urls))
                for links in all_urls:
                    print(links)
                    inindex.append([query, links, score])
                savesqllist(inindex)
                # удаляем из таблицы ключ
                delete_keyword = f"DELETE FROM keywords WHERE fraza = '{query}'"
                execute_query(connection, delete_keyword)

            time.sleep(1)
            driver.close()
            driver.quit()
            return
    except:
        driver.close()
        driver.quit()
        time.sleep(10)
        print('Перезапуск get_driver(query, score)')
        get_driver(query, score)
        pass
def read_queries_list_from_file(filename):
    f = open( filename, mode = 'r', encoding = 'utf-8' )    
    return [x.strip() for x in f]

def getkeys(keywords):
    delete_grup = "DELETE FROM errkeys"
    execute_query(connection, delete_grup)
    i = 0
    for row in keywords:
        clear_output(wait=True)
        i += 1
        print('Done -', i)
        query = row[1]
        score = row[2]
        print(query, score)
        get_driver(query, score)
        
        
def main():
    global fil
    fil = str(f'google.{country}_{dt}.csv')
    # получаем прокси
    proxies = listOfproxy(fileproxy)

    select_keywords = "SELECT * from keywords"
    keywords = execute_read_query(connection, select_keywords)
    getkeys(keywords)
    # пороверка на ошибки сбора
    select_keywords = "SELECT * from errkeys"
    keywords = execute_read_query(connection, select_keywords)
    if keywords:
        print("Досбор")
        getkeys(keywords)
    select_keywords = "SELECT * from errkeys"
    keywords = execute_read_query(connection, select_keywords)

    print('Не собрано', len(keywords), 'Done')
    
#Кластеризатор из базы данных + value Midle       
def midle():        
    delete_grup = "DELETE FROM grup"
    execute_query(connection, delete_grup)

    clasterKeys = []
    clasterKey = []
    typeCl = 'Midle'

    select_keywords = "SELECT * from keywords2"
    keywords = execute_read_query(connection, select_keywords)

    top_urls = "SELECT * from top"
    top_urls = execute_read_query(connection, top_urls)

    for i in tqdm(keywords):
    #     clear_output(wait=True)
        keyword = i[1]
        if keyword in clasterKeys: continue

        clasterKeyM = []
        clasterKeyM = list(map(lambda x: x[2], list(filter(lambda x: x[1] == keyword, top_urls))))


        for h in keywords:
    #         clear_output(wait=True)
            keynext = h[1]
            value = h[2]
            if keynext in clasterKeys: continue

            clasterKeyD = []
            clasterKeyD = list(map(lambda x: x[2], list(filter(lambda x: x[1] == keynext, top_urls))))

            result=list(set(clasterKeyM) & set(clasterKeyD)) 

            if len(result) > 2:

                clasterKeys.append(keynext)


                create_groop = f"""
                INSERT INTO
                  `grup` (`grope`, `keyw`, `score`)
                VALUES
                  ('{keyword}', '{keynext}', '{value}');
                """

                execute_query(connection, create_groop)

    # Сохраняем кластеры

    addCluster = []
    cluster = ('Grope', 'Keys', 'Value')
    addCluster.append(cluster)

    select_users = "SELECT * from grup"
    keywords = execute_read_query(connection, select_users)


    for result in keywords:
        groupTop = result[1]
        keywordTop = result[2]
        valueTop = result[3]
        cluster = (groupTop, keywordTop, valueTop)
        addCluster.append(cluster)

    with xlsxwriter.Workbook(f'{dt}_clusters_{typeCl}.xlsx') as workbook:
        worksheet = workbook.add_worksheet()
        for row_num, data in enumerate(addCluster):
            worksheet.write_row(row_num, 0, data)
    print('Done')

    # Відправляємо повідомлення в телеграм і файл

    try:
        chatId = chat_id
        send = 'Кластеризація SOFT завершена'
        bot.send_message(chatId, send)
    except:
        print("Відправити повідомлення не вдалося")
    try:
        doc = open(f'{dt}_clusters_{typeCl}.xlsx', 'rb')
        bot.send_document(chatId, doc) 
    except:
        print("Відправити файл не вдалося")
#Кластеризатор из базы данных + value Midle+Soft
def soft():
    delete_grup = "DELETE FROM grup"
    execute_query(connection, delete_grup)

    clasterKeys = []
    clasterKey = []
    typeCl = 'Midle_Soft'


    select_keywords = "SELECT * from keywords2"
    keywords = execute_read_query(connection, select_keywords)

    top_urls = "SELECT * from top"
    top_urls = execute_read_query(connection, top_urls)

    for i in tqdm(keywords):
        clasterKeysGroup = []
    #     clear_output(wait=True)
        keyword = i[1]
        if keyword in clasterKeys: continue
        #Урли материнської групи
        clasterKeyM = []
        clasterKeyM = list(map(lambda x: x[2], list(filter(lambda x: x[1] == keyword, top_urls))))
        clasterKeysGroup.append(keyword)

        for h in keywords:
    #         clear_output(wait=True)
            keynext = h[1]
            value = h[2]
            if keynext in clasterKeys: continue
            #Урли дочірньої групи
            clasterKeyD = []
            clasterKeyD = list(map(lambda x: x[2], list(filter(lambda x: x[1] == keynext, top_urls))))

            result=list(set(clasterKeyM) & set(clasterKeyD)) 

            if len(result) > 2:
                clasterKeys.append(keynext)
                clasterKeysGroup.append(keynext)

                create_groop = f"""
                INSERT INTO
                  `grup` (`grope`, `keyw`, `score`)
                VALUES
                  ('{keyword}', '{keynext}', '{value}');
                """

                execute_query(connection, create_groop)

                for j in keywords:
                    lastkeyword = clasterKeysGroup[-1]
                    clasterKeyC = []
                    clasterKeyC = list(map(lambda x: x[2], list(filter(lambda x: x[1] == lastkeyword, top_urls))))
            #         clear_output(wait=True)
                    keynext = j[1]
                    value = j[2]
                    if keynext in clasterKeys: continue

                    clasterKeyE = []
                    clasterKeyE = list(map(lambda x: x[2], list(filter(lambda x: x[1] == keynext, top_urls))))

                    result=list(set(clasterKeyC) & set(clasterKeyE)) 

                    if len(result) > 2:

                        clasterKeys.append(keynext)
                        clasterKeysGroup.append(keynext)


                        create_groop = f"""
                        INSERT INTO
                          `grup` (`grope`, `keyw`, `score`)
                        VALUES
                          ('{keyword}', '{keynext}', '{value}');
                        """

                        execute_query(connection, create_groop)

    # Сохраняем кластеры

    dt = datetime.datetime.now().strftime("%d_%m_%Y_%H_%M")

    addCluster = []
    cluster = ('Grope', 'Keys', 'Value')
    addCluster.append(cluster)

    select_users = "SELECT * from grup"
    keywords = execute_read_query(connection, select_users)


    for result in keywords:
        groupTop = result[1]
        keywordTop = result[2]
        valueTop = result[3]
        cluster = (groupTop, keywordTop, valueTop)
        addCluster.append(cluster)

    with xlsxwriter.Workbook(f'{dt}_clusters_{typeCl}.xlsx') as workbook:
        worksheet = workbook.add_worksheet()
        for row_num, data in enumerate(addCluster):
            worksheet.write_row(row_num, 0, data)
    print('Done')

    # Відправляємо повідомлення в телеграм і файл

    try:
        chatId = chat_id
        send= 'Кластеризація SOFT завершена'
        bot.send_message(chatId, send)
    except:
        print("Відправити повідомлення не вдалося")
    try:
        doc = open(f'{dt}_clusters_{typeCl}.xlsx', 'rb')
        bot.send_document(chatId, doc) 
    except:
        print("Відправити файл не вдалося")
        
# Сохраняем ТОП в файл
def get_top():
    fil = dt+"_top.csv"

    addCluster = []
    cluster = 'Grope@Keys'
    addCluster.append(cluster)

    select_users = "SELECT * from top"
    keywords = execute_read_query(connection, select_users)


    for result in keywords:
        groupTop = result[1]
        keywordTop = result[2]
        cluster = f'{groupTop}@{keywordTop}'
        addCluster.append(cluster)

    def savefile(addCluster):
        with open(fil, 'a', encoding="utf-8") as f:
            w = csv.writer(f, lineterminator='\n')
            for item in addCluster:
                w.writerow([item])
    #     time.sleep(2)
        f.close()
    savefile(addCluster)
    try:
        doc = open(fil, 'rb')
        bot.send_document(chatId, doc) 
    except:
        print("Відправити не вдалося")
    import pandas
    try:
        file_read = pandas.read_csv(fil, delimiter='@')
        file_read.to_excel(dt+"_top.xlsx", index=None, header=True)
        print('xls збережено')  
        try:
            # send= 'Кластеризація SOFT завершена'
            # bot.send_message(chatId, send)
            doc = open(f'{dt}_top.xlsx', 'rb')
            bot.send_document(chatId, doc) 
        except:
            print("Відправити не вдалося")

    except:
        print("Конвертувати в xls не вдалося")        
    
# создаем базу данных название базы данных dt
def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection


def takeData():
#     dt = datetime.datetime.now().strftime("%d_%m_%Y_%H_%M")
#     print ('Data -',dt)
    return datetime.datetime.now().strftime("%d_%m_%Y_%H_%M")

def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path, check_same_thread = False)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection
# Робота з таблицями бази даних
def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as e:
        print(f"The error '{e}' occurred")

def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
#         print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")
        
#Обробка файлу з ключами        
def getfile(filekeys):
    with open(filekeys, encoding="utf8") as csvfile:
        spamreader = csv.reader(csvfile, delimiter=";")
        next(spamreader)
        for row in tqdm(spamreader):
            clear_output(wait=True)
            keyword = row[0].strip().replace('\ufeff', '')
            value = int(row[1].strip().replace('\ufeff', ''))

            create_keyword = f"""
            INSERT OR IGNORE INTO
              keywords (fraza, score)
            VALUES
              ('{keyword}', '{value}');
            """

            execute_query(connection, create_keyword)  
    csvfile.close()
    
def getkeywords(filekeys):
#     t0 = time()  
#     file = 'keys-for-cluster_ua.csv'
    # file = 'keys-for-top-city.csv'
    getfile(filekeys)
    #Копируем таблицу в keywords2
    copy_table = "INSERT INTO keywords2 SELECT * FROM keywords"
    keywords = execute_read_query(connection, copy_table)
#     allTime = int(time() - t0)
#     print(f'Таблиця2 скопійована. Наповнення завершене за {allTime} секунд')
    print(f'Таблиця2 скопійована. Наповнення завершене')

def create_all_tables():         
    create_top_table = """
    CREATE TABLE IF NOT EXISTS top (
      id INTEGER PRIMARY KEY AUTOINCREMENT,
      keywd TEXT NOT NULL,
      url TEXT NOT NULL,
      score INTEGER
    );
    """
    execute_query(connection, create_top_table)

    create_keywords_table = """
    CREATE TABLE IF NOT EXISTS keywords (
      id INTEGER PRIMARY KEY AUTOINCREMENT,
      fraza TEXT NOT NULL UNIQUE,
      score INTEGER
    );
    """
    execute_query(connection, create_keywords_table) 

    create_keywords_table = """
    CREATE TABLE IF NOT EXISTS keywords2 (
      id INTEGER PRIMARY KEY AUTOINCREMENT,
      fraza TEXT NOT NULL UNIQUE,
      score INTEGER
    );
    """
    execute_query(connection, create_keywords_table) 

    create_group_table = """
    CREATE TABLE IF NOT EXISTS grup (
      id INTEGER PRIMARY KEY AUTOINCREMENT,
      grope TEXT NOT NULL,
      keyw TEXT NOT NULL,
      score INTEGER
    );
    """
    execute_query(connection, create_group_table)

    create_err_table = """
    CREATE TABLE IF NOT EXISTS errkeys (
      id INTEGER PRIMARY KEY AUTOINCREMENT,
      errkey TEXT NOT NULL,
      score INTEGER
    );
    """
    execute_query(connection, create_err_table)

# Дані Гео для парсингу
def en_US():
    global uule,hl,country,num,zone
    hl = 'en' #hl=sv          Swedish https://en.wikipedia.org/wiki/List_of_ISO_639-1_codes
    country = 'us' #se 	Sweden  
    num = '10' # Количество результатов поиска
    zone = 'com' #'com.ua'
    region = 'United States'  #'Kyiv,Kyiv city,Ukraine', 'UA'   New York,New York,United States   Moscow,Moscow,Russia
    uule = uule_grabber.uule(region)
    print(uule)
    
def ru_UA():
    global uule,hl,country,num,zone
    hl = 'ru' #hl=sv          Swedish https://en.wikipedia.org/wiki/List_of_ISO_639-1_codes
    country = 'ua' #se 	Sweden  
    num = '10' # Количество результатов поиска
    zone = 'com.ua' #'com.ua'
    region = 'Kyiv,Kyiv city,Ukraine'  #'Kyiv,Kyiv city,Ukraine', 'UA'   New York,New York,United States   Moscow,Moscow,Russia
    uule = uule_grabber.uule(region)
    print(uule)
    
def en_US():
    global uule,hl,country,num,zone
    hl = 'en' #hl=sv          Swedish https://en.wikipedia.org/wiki/List_of_ISO_639-1_codes
    country = 'us' #se 	Sweden  
    num = '10' # Количество результатов поиска
    zone = 'com' #'com.ua'
    region = 'United States'  #'Kyiv,Kyiv city,Ukraine', 'UA'   New York,New York,United States   Moscow,Moscow,Russia
    uule = uule_grabber.uule(region)
    print(uule)
    
def stop():    
    connection.close()
    os.remove("{}.sqlite".format(dt))
# Обработчик текстовых сообщений к боту. 
@bot.message_handler(content_types=['text'])
def get_text_messages(message): 
    chat_Id = message.chat.id

    #print(message.chat.id) - чтобы узнать id чата

    if message.text == "/help":
        bot.send_message(message.chat.id,"Доступні наступні команди:")
        bot.send_message(message.chat.id,"/start - Підготовка бази даних \n")
        
    elif message.text == "/start":
        global dt, connection
        dt = takeData()
        bot.send_message(message.chat.id,f"База даних, {dt}\n")
        connection = create_connection(r"G:\Yura\Python\Парсер G-Y\{}.sqlite".format(dt))
        bot.send_message(message.chat.id,f"Таблиці БД створені\n")
        create_all_tables()
        bot.send_message(message.chat.id,"Очікую на файл з ключами .csv utf-8 delimiter ; \n")

    elif message.text == "/take_keywords":        
        if len(filekeys)>5:
            bot.send_message(message.chat.id,f"Наповнюю таблицю ключами\n")
            getkeywords(filekeys)
            bot.send_message(message.chat.id,f"таблиця наповнена\n")
            bot.send_message(message.chat.id,f"Оберіть регіон /take_geo\n")
        else:
            bot.send_message(message.chat.id,f"Потрібен файл з ключами\n")
    elif message.text == "/take_geo":
        bot.send_message(message.chat.id,f"/en_US - Обираємо Google USA\n/ru_UA - Обираємо Google UA\n/ru_RU - Обираємо Google RU\n")
    elif message.text == "/en_US":
        en_US()
        bot.send_message(message.chat.id,f"Прараметри парсера \nuule - {uule}\n hl - {hl}\n country - {country}\n num - {num}\n zone - {zone}\n")
        bot.send_message(message.chat.id,f"Для запуску парсера тисни /start_parser\n")
    elif message.text == "/ru_UA":
        ru_UA()
        bot.send_message(message.chat.id,f"Прараметри парсера \nuule - {uule}\n hl - {hl}\n country - {country}\n num - {num}\n zone - {zone}\n")
        bot.send_message(message.chat.id,f"Для запуску парсера тисни /start_parser\n")
    elif message.text == "/ru_RU":
        ru_RU()
        bot.send_message(message.chat.id,f"Прараметри парсера \nuule - {uule}\n hl - {hl}\n country - {country}\n num - {num}\n zone - {zone}\n")
        bot.send_message(message.chat.id,f"Для запуску парсера тисни /start_parser\n")
        
            
    elif message.text == "/start_parser":
        bot.send_message(message.chat.id,f"Старт парсера\n")
        main()
        bot.send_message(message.chat.id,f"Парсер роботу закінчив\n")
        bot.send_message(message.chat.id,f"Отримати зібраний топ оберіть /get_top\n Запустити кластеризацію, оберіть:\n - /midle \n - /soft\n")
    elif message.text == "/get_top":
        bot.send_message(message.chat.id,f"Старт Midle кластеризації\n")
        get_top()        
    elif message.text == "/midle":
        bot.send_message(message.chat.id,f"Старт Midle кластеризації\n")
        midle()
        bot.send_message(message.chat.id,f"Зупинити роботу скрипта та видалити поточну базу даних натисни /stop\n")
    elif message.text == "/soft":
        bot.send_message(message.chat.id,f"Старт Soft кластеризації\n")
        soft()
        bot.send_message(message.chat.id,f"Зупинити роботу скрипта та видалити поточну базу даних натисни /stop\n")
    elif message.text == "/stop":
        stop()
        bot.send_message(message.chat.id,f"Робота скрипта зупинена, база даних '{dt}' видалена\n")
    else:
        bot.send_message(message.chat.id, "Привіт, список команд '/help'")

        
@bot.message_handler(content_types=['document'])
def handle_docs_photo(message):
    try:
        global chat_id, filekeys
        chat_id = message.chat.id

        file_info = bot.get_file(message.document.file_id)
        downloaded_file = bot.download_file(file_info.file_path)
        

        src = 'G:/Yura/Python/Парсер G-Y/' + message.document.file_name; #G:\Yura\Python\Парсер G-Y
        filekeys = message.document.file_name
        with open(src, 'wb') as new_file:
            new_file.write(downloaded_file)

        bot.reply_to(message, f"Файл {filekeys} збережений")
        bot.send_message(message.chat.id,f"Для наповнення таблиці натисни /take_keywords\n")
    except Exception as e:
        bot.reply_to(message, e)
bot.polling(none_stop=True, interval=0)        